<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Neural Collaborative Filtering (NCF)

This notebook implements Neural Collaborative Filtering (NCF), which is an innovative algorithm based on deep neural networks to tackle the key problem in recommendation — collaborative filtering — on the basis of implicit feedback.

## 0 Global Settings and Imports

In [ ]:
%pip install recommenders
import recommenders

  Using cached recommenders-1.2.1-py3-none-any.whl.metadata (13 kB)
  Using cached category_encoders-2.9.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached cornac-2.3.5-cp312-cp312-manylinux1_x86_64.whl.metadata (51 kB)
  Using cached locust-2.42.6-py3-none-any.whl.metadata (10 kB)
  Using cached memory_profiler-0.61.0-py3-none-any.whl.metadata (20 kB)
  Using cached retrying-1.4.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.3/355.3 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.3 MB/s eta 0:00

In [ ]:
!pip install "numpy<2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 132.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cornac 2.3.5 requires numpy>2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0

In [ ]:
!pip install surprise

from surprise import KNNBasic
from surprise import Dataset
from surprise.reader import Reader

In [ ]:
import sys
import os
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages
from recommenders.datasets.python_splitters import python_random_split
from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                     recall_at_k, get_top_k_items)
from recommenders.utils.constants import SEED as DEFAULT_SEED

from sklearn import preprocessing

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))


System version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas version: 2.2.2


/usr/local/lib/python3.12/dist-packages/pandera/_pandas_deprecated.py:157: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [ ]:
# top k items to recommend
TOP_K = 10

# Model parameters
EPOCHS = 100
BATCH_SIZE = 256

SEED = DEFAULT_SEED  # Set None for non-deterministic results

## Read Data

In [ ]:
import pandas as pd


df=pd.read_csv("/content/collected data.csv")
df.rename(columns = {'user_id':'userID', 'restaurant_id':'itemID'}, inplace = True)

df

Mounted at /content/drive


,userID,City,User Longitude,User Latitude,restaurant,Resturant Longitude,Resturant Latitude,Categories,Special Diets,rating,itemID
0,16,Jeddah,39.279395,21.167678,Khayal Restaurant,39.141060,21.609583,"Barbecue, Middle Eastern","Vegetarian Friendly, Vegan Options",3.6,0
1,38,Jeddah,39.262074,21.036712,Khayal Restaurant,39.141060,21.609583,"Barbecue, Middle Eastern","Vegetarian Friendly, Vegan Options",3.6,0
2,46,Jeddah,39.127784,21.175333,Khayal Restaurant,39.141060,21.609583,"Barbecue, Middle Eastern","Vegetarian Friendly, Vegan Options",4.9,0
3,72,Jeddah,39.240627,21.630606,Khayal Restaurant,39.141060,21.609583,"Barbecue, Middle Eastern","Vegetarian Friendly, Vegan Options",4.7,0
4,90,Jeddah,39.364720,21.865419,Khayal Restaurant,39.141060,21.609583,"Barbecue, Middle Eastern","Vegetarian Friendly, Vegan Options",5.0,0
...,...,...,...,...,...,...,...,...,...,...,...
6471,2319,Eastern Province,53.857231,24.191063,Hamra Chicken,50.204263,26.290657,"Lebanese, Fast Food, Mediterranean, Middle Eas...",Non,4.1,33
6472,2322,Eastern Province,53.857231,24.191063,Hamra Chicken,50.204263,26.290657,"Lebanese, Fast Food, Mediterranean, Middle Eas...",Non,4.4,33
6473,2324,Eastern Province,53.857231,24.191063,Hamra Chicken,50.204263,26.290657,"Lebanese, Fast Food, Mediterranean, Middle Eas...",Non,3.4,33
6474,2326,Eastern Province,53.857231,24.191063,Hamra Chicken,50.204263,26.290657,"Lebanese, Fast Food, Mediterranean, Middle Eas...",Non,3.5,33


In [ ]:
unique_user_ids = df['userID'].unique()
unique_item_ids = df['itemID'].unique()
print(len(unique_user_ids))
print(len(unique_item_ids))

1062
34


In [ ]:
# Encode user and item IDs
user_encoder = preprocessing.LabelEncoder()
df['userID'] = user_encoder.fit_transform(df['userID'])

item_encoder = preprocessing.LabelEncoder()
df['itemID'] = item_encoder.fit_transform(df['itemID'])

df.head()

,userID,City,User Longitude,User Latitude,restaurant,Resturant Longitude,Resturant Latitude,Categories,Special Diets,rating,itemID
0,15,Jeddah,39.279395,21.167678,Khayal Restaurant,39.14106,21.609583,"Barbecue, Middle Eastern","Vegetarian Friendly, Vegan Options",3.6,0
1,32,Jeddah,39.262074,21.036712,Khayal Restaurant,39.14106,21.609583,"Barbecue, Middle Eastern","Vegetarian Friendly, Vegan Options",3.6,0
2,38,Jeddah,39.127784,21.175333,Khayal Restaurant,39.14106,21.609583,"Barbecue, Middle Eastern","Vegetarian Friendly, Vegan Options",4.9,0
3,61,Jeddah,39.240627,21.630606,Khayal Restaurant,39.14106,21.609583,"Barbecue, Middle Eastern","Vegetarian Friendly, Vegan Options",4.7,0
4,76,Jeddah,39.364720,21.865419,Khayal Restaurant,39.14106,21.609583,"Barbecue, Middle Eastern","Vegetarian Friendly, Vegan Options",5.0,0


## Holdout Validation

In [ ]:
from recommenders.datasets.python_splitters import python_stratified_split
train, test = python_stratified_split(df, ratio=0.75, filter_by='user', col_user='userID', col_item='itemID', seed=42)


Filter out any users or items in the test set that do not appear in the training set.

In [ ]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

Create a test set containing the last interaction for each user as for the leave-one-out evaluation.

In [ ]:
leave_one_out_test = test.groupby("userID").last().reset_index()

In [ ]:
leave_one_out_train = train.groupby("userID").last().reset_index()

In [ ]:
leave_one_out_train

,userID,City,User Longitude,User Latitude,restaurant,Resturant Longitude,Resturant Latitude,Categories,Special Diets,rating,itemID
0,0,Jeddah,39.289351,21.017256,Wakame,39.127817,21.577143,"Japanese, Seafood, Sushi, Asian","Vegetarian Friendly, Vegan Options",4.8,27
1,1,Jeddah,39.104774,21.500767,Sura Korean Fine Dining,39.153422,21.558961,"Asian, Korean","Vegetarian Friendly, Vegan Options",4.5,25
2,2,Jeddah,39.106624,21.665551,Myazu,39.152924,21.550449,"Japanese, Sushi, Asian","Vegetarian Friendly, Vegan Options, Gluten Fre...",3.7,12
3,3,Riyadh,46.640149,24.590036,La Brasserie,46.684105,24.690431,"French, International","Vegetarian Friendly, Vegan Options",4.5,9
4,4,Riyadh,46.596925,24.714269,Lusin,46.683451,24.697821,"Mediterranean, Middle Eastern, Armenian","Vegetarian Friendly, Vegan Options, Gluten Fre...",3.8,19
...,...,...,...,...,...,...,...,...,...,...,...
1057,1057,Jeddah,39.196930,21.734118,Sura Korean Fine Dining,39.153422,21.558961,"Asian, Korean","Vegetarian Friendly, Vegan Options",4.1,25
1058,1058,Eastern Province,53.857231,24.191063,Hamra Chicken,50.204263,26.290657,"Lebanese, Fast Food, Mediterranean, Middle Eas...",Non,3.5,33
1059,1059,Eastern Province,54.645956,26.594969,Abu Nawas,50.082476,26.439792,"Lebanese, Mediterranean, Middle Eastern",Vegetarian Friendly,4.1,24
1060,1060,Jeddah,39.365049,21.426655,Wakame,39.127817,21.577143,"Japanese, Seafood, Sushi, Asian","Vegetarian Friendly, Vegan Options",3.6,27


In [ ]:
train_file = "./train.csv"
test_file = "./test.csv"
leave_one_out_test_file = "./leave_one_out_test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)
leave_one_out_test.to_csv(leave_one_out_test_file, index=False)

In [ ]:
leave_one_out_train.to_csv(train_file, index=False)


Write datasets to csv files.

In [ ]:
data = NCFDataset(train_file=train_file, test_file=leave_one_out_test_file, seed=SEED, overwrite_test_file_full=True)

  0%|          | 0/794 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.12/dis

Train NCF based on TensorFlow
The NCF has a lot of parameters. The most important ones are:

`n_factors`, which controls the dimension of the latent space. Usually, the quality of the training set predictions grows with as n_factors gets higher.

`layer_sizes`, sizes of input layer (and hidden layers) of MLP, input type is list.

`n_epochs`, which defines the number of iteration of the SGD procedure.
Note that both parameter also affect the training time.

`model_type`, we can train single `"MLP"`, `"GMF"` or combined model `"NCF"` by changing the type of model.

We will here set `n_factors` to 4, `layer_sizes` to `[16,8,4]`,  `n_epochs` to 100, `batch_size` to 256. To train the model, we simply need to call the `fit()` method.

In [ ]:

model = NCF (
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=8,
    layer_sizes=[64, 32, 16],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=0.0001,
    verbose=10,
    seed=SEED
)

/usr/local/lib/python3.12/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:

with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

Took 160.62586420999997 seconds for training.


### Prediction and Evaluation

#### Prediction

Now that our model is fitted, we can call `predict` to get some `predictions`. `predict` returns an internal object Prediction which can be easily converted back to a dataframe:

In [ ]:
predictions = [[int(row.userID), int(row.itemID), model.predict(int(row.userID), int(row.itemID))]
               for (_, row) in test.iterrows()]


predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()



,userID,itemID,prediction
0,1,2,0.198573
1,8,29,0.216125
2,9,1,0.189587
3,10,17,0.202765
4,16,20,0.200050


#### Generic Evaluation
To compute ranking metrics, we need predictions on all user, item pairs. We remove though the items already rated by the user, since we choose not to recommend them again.

In [ ]:

with Timer() as test_time:

    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time.interval))

Took 1.057120939000015 seconds for prediction.


## k-fold Cross Validation

### Prediction and Evaluation

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*DataFrame.swapaxes.*")

# define a cross-validation iterator
#cross val
from surprise.model_selection import cross_validate
from surprise import accuracy
import pandas as pd # Import pandas
import numpy as np # Import numpy for std calculation
# You can use recommenders evaluation functions if you convert Surprise predictions to a pandas DataFrame
from recommenders.evaluation.python_evaluation import (
        rmse, mae, precision_at_k, recall_at_k, ndcg_at_k, map_at_k,rsquared,exp_var
    )


reader = Reader(rating_scale=(1, 5))  # Adjust based on your rating scale

# Convert the full 'data' DataFrame to a Surprise Dataset object
surprise_data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

import recommenders
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions
from surprise.model_selection import KFold


kf = KFold(n_splits=10)

# Lists to store metrics for each fold
rmse_scores = []
mae_scores = []
rsquared_scores = []
exp_var_scores = []
map_scores = []
ndcg_scores = []
precision_scores = []
recall_scores = []

for trainset_surprise, testset_surprise in kf.split(surprise_data):

    # Convert Surprise trainset and testset to DataFrames
    train_df = pd.DataFrame([(uid, iid, rating) for (uid, iid, rating) in
                           trainset_surprise.build_testset()],
                          columns=['userID', 'itemID', 'rating'])

    test_df = pd.DataFrame([(uid, iid, rating) for (uid, iid, rating) in
                          testset_surprise],
                         columns=['userID', 'itemID', 'rating'])

    # Combine train and test for the current fold
    combined_df = pd.concat([train_df, test_df], ignore_index=True)

    # Identify the last interaction for each user as test set
    leave_one_out_test_df = combined_df.sort_values(['userID', 'rating']).groupby('userID').tail(1)

    # Create training set by removing the test interactions
    train_df_ncf = combined_df.merge(leave_one_out_test_df[['userID', 'itemID']],
                                   on=['userID', 'itemID'],
                                   how='left',
                                   indicator=True)
    train_df_ncf = train_df_ncf[train_df_ncf['_merge'] == 'left_only'].drop('_merge', axis=1)

    # Remove users from leave_one_out_test_df if they have no interactions in train_df_ncf
    users_with_train_data = train_df_ncf['userID'].unique()
    leave_one_out_test_df = leave_one_out_test_df[leave_one_out_test_df['userID'].isin(users_with_train_data)]

    # Ensure consistency: also remove items from test that don't exist in train
    items_with_train_data = train_df_ncf['itemID'].unique()
    leave_one_out_test_df = leave_one_out_test_df[leave_one_out_test_df['itemID'].isin(items_with_train_data)]

    # Sort the training data by userID to satisfy NCFDataset requirements
    train_df_ncf_sorted = train_df_ncf.sort_values('userID').reset_index(drop=True)

    print(f"Fold stats: Train users: {train_df_ncf_sorted['userID'].nunique()}, "
          f"Test users: {leave_one_out_test_df['userID'].nunique()}, "
          f"Train interactions: {len(train_df_ncf_sorted)}, "
          f"Test interactions: {len(leave_one_out_test_df)}")

    # Save to files for NCFDataset
    train_file = "./train.csv"
    test_file = "./leave_one_out_test.csv"

    # Save sorted training data
    train_df_ncf_sorted.to_csv(train_file, index=False)
    leave_one_out_test_df.to_csv(test_file, index=False)

    # Initialize and train NCF model
    data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED, overwrite_test_file_full=True)
    model = NCF (
        n_users=data.n_users,
        n_items=data.n_items,
        model_type="NeuMF",
        n_factors=8,
        layer_sizes=[64, 32, 16],
        n_epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        learning_rate=0.0001,
        verbose=10,
        seed=SEED
    )

    with Timer() as train_time:
        model.fit(data)

    print("Took {} seconds for training.".format(train_time.interval))

    # Generate predictions for the leave-one-out test set
    predictions = [[int(row.userID), int(row.itemID), model.predict(int(row.userID), int(row.itemID))]
                  for (_, row) in leave_one_out_test_df.iterrows()]

    predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])

    # Generate all predictions for ranking metrics
    with Timer() as test_time:
        users, items, preds = [], [], []
        items_list = list(train_df_ncf_sorted.itemID.unique())
        for user in train_df_ncf_sorted.userID.unique():
            user_list = [user] * len(items_list)
            users.extend(user_list)
            items.extend(items_list)
            preds.extend(list(model.predict(user_list, items_list, is_list=True)))

        all_predictions = pd.DataFrame(data={"userID": users, "itemID": items, "prediction": preds})

        # Remove interactions that are in training set
        merged = pd.merge(train_df_ncf_sorted, all_predictions, on=["userID", "itemID"], how="outer")
        all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

    print("Took {} seconds for prediction.".format(test_time.interval))

    # Calculate evaluation metrics
    eval_rmse = rmse(leave_one_out_test_df, predictions)
    eval_mae = mae(leave_one_out_test_df, predictions)
    eval_rsquared = rsquared(leave_one_out_test_df, predictions)
    eval_exp_var = exp_var(leave_one_out_test_df, predictions)

    eval_map = map_at_k(leave_one_out_test_df, all_predictions, col_prediction='prediction', k=TOP_K)
    eval_ndcg = ndcg_at_k(leave_one_out_test_df, all_predictions, col_prediction='prediction', k=TOP_K)
    eval_precision = precision_at_k(leave_one_out_test_df, all_predictions, col_prediction='prediction', k=TOP_K)
    eval_recall = recall_at_k(leave_one_out_test_df, all_predictions, col_prediction='prediction', k=TOP_K)

    # Store metrics
    rmse_scores.append(eval_rmse)
    mae_scores.append(eval_mae)
    rsquared_scores.append(eval_rsquared)
    exp_var_scores.append(eval_exp_var)
    map_scores.append(eval_map)
    ndcg_scores.append(eval_ndcg)
    precision_scores.append(eval_precision)
    recall_scores.append(eval_recall)

    print("Microsoft Recommenders Evaluation:")
    print("RMSE:\t\t%f" % eval_rmse,
          "MAE:\t\t%f" % eval_mae,
          "rsquared:\t%f" % eval_rsquared,
          "exp var:\t%f" % eval_exp_var, sep='\n')

    print('----')

    print("MAP:\t%f" % eval_map,
          "NDCG:\t%f" % eval_ndcg,
          "Precision@K:\t%f" % eval_precision,
          "Recall@K:\t%f" % eval_recall, sep='\n')
    print("\n" + "="*50 + "\n")

print("\n=== Average Metrics Across Folds ===")

metrics_data = {
    "Metric": [
        "RMSE",
        "MAE",
        "R-squared",
        "Explained Variance",
        f"MAP@{TOP_K}",
        f"NDCG@{TOP_K}",
        f"Precision@{TOP_K}",
        f"Recall@{TOP_K}"
    ],
    "Value": [
        f"{np.mean(rmse_scores):.4f} \u00B1 {np.std(rmse_scores):.4f}",
        f"{np.mean(mae_scores):.4f} \u00B1 {np.std(mae_scores):.4f}",
        f"{np.mean(rsquared_scores):.4f} \u00B1 {np.std(rsquared_scores):.4f}",
        f"{np.mean(exp_var_scores):.4f} \u00B1 {np.std(exp_var_scores):.4f}",
        f"{np.mean(map_scores):.4f} \u00B1 {np.std(map_scores):.4f}",
        f"{np.mean(ndcg_scores):.4f} \u00B1 {np.std(ndcg_scores):.4f}",
        f"{np.mean(precision_scores):.4f} \u00B1 {np.std(precision_scores):.4f}",
        f"{np.mean(recall_scores):.4f} \u00B1 {np.std(recall_scores):.4f}"
    ]
}

df_metrics = pd.DataFrame(metrics_data)
display(df_metrics)

print(f"\n=== Average Metrics (Mean \u00B1 Std) ===")
print(f"RMSE: {np.mean(rmse_scores):.4f} \u00B1 {np.std(rmse_scores):.4f}")
print(f"MAE: {np.mean(mae_scores):.4f} \u00B1 {np.std(mae_scores):.4f}")
print(f"R-squared: {np.mean(rsquared_scores):.4f} \u00B1 {np.std(rsquared_scores):.4f}")
print(f"Explained Variance: {np.mean(exp_var_scores):.4f} \u00B1 {np.std(exp_var_scores):.4f}")
print(f"MAP@{TOP_K}: {np.mean(map_scores):.4f} \u00B1 {np.std(map_scores):.4f}")
print(f"NDCG@{TOP_K}: {np.mean(ndcg_scores):.4f} \u00B1 {np.std(ndcg_scores):.4f}")
print(f"Precision@{TOP_K}: {np.mean(precision_scores):.4f} \u00B1 {np.std(precision_scores):.4f}")
print(f"Recall@{TOP_K}: {np.mean(recall_scores):.4f} \u00B1 {np.std(recall_scores):.4f}")

Fold stats: Train users: 930, Test users: 930, Train interactions: 5414, Test interactions: 930


100%|██████████| 930/930 [00:04<00:00, 206.19it/s]
/usr/local/lib/python3.12/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Took 158.07877753399998 seconds for training.
Took 0.9472893419999764 seconds for prediction.
Microsoft Recommenders Evaluation:
RMSE:		4.224372
MAE:		4.207868
rsquared:	-158.736150
exp var:	-0.245655
----
MAP:	0.138364
NDCG:	0.264986
Precision@K:	0.037097
Recall@K:	0.741935


Fold stats: Train users: 930, Test users: 930, Train interactions: 5414, Test interactions: 930


100%|██████████| 930/930 [00:02<00:00, 362.29it/s]
/usr/local/lib/python3.12/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Took 158.92303414899993 seconds for training.
Took 0.925397342999986 seconds for prediction.
Microsoft Recommenders Evaluation:
RMSE:		4.223212
MAE:		4.206633
rsquared:	-158.648415
exp var:	-0.250988
----
MAP:	0.139733
NDCG:	0.266399
Precision@K:	0.037366
Recall@K:	0.747312


Fold stats: Train users: 930, Test users: 930, Train interactions: 5414, Test interactions: 930


...

Took 157.99358557999994 seconds for training.
Took 0.641781044000254 seconds for prediction.
Microsoft Recommenders Evaluation:
RMSE:		4.223690
MAE:		4.207138
rsquared:	-158.684581
exp var:	-0.249140
----
MAP:	0.135771
NDCG:	0.259408
Precision@K:	0.036183
Recall@K:	0.723656


Fold stats: Train users: 930, Test users: 930, Train interactions: 5414, Test interactions: 930


100%|██████████| 930/930 [00:02<00:00, 430.78it/s]
/usr/local/lib/python3.12/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Took 153.64186947999997 seconds for training.
Took 0.8271116420000908 seconds for prediction.
Microsoft Recommenders Evaluation:
RMSE:		4.222008
MAE:		4.205871
rsquared:	-158.557399
exp var:	-0.217323
----
MAP:	0.135134
NDCG:	0.261566
Precision@K:	0.036882
Recall@K:	0.737634



=== Average Metrics Across Folds ===


,Metric,Value
0,RMSE,4.2217 ± 0.0022
1,MAE,4.2053 ± 0.0022
2,R-squared,-158.5314 ± 0.1682
3,Explained Variance,-0.2331 ± 0.0178
4,MAP@20,0.1376 ± 0.0018
5,NDCG@20,0.2632 ± 0.0019
6,Precision@20,0.0368 ± 0.0005
7,Recall@20,0.7361 ± 0.0099



=== Average Metrics (Mean ± Std) ===
RMSE: 4.2217 ± 0.0022
MAE: 4.2053 ± 0.0022
R-squared: -158.5314 ± 0.1682
Explained Variance: -0.2331 ± 0.0178
MAP@20: 0.1376 ± 0.0018
NDCG@20: 0.2632 ± 0.0019
Precision@20: 0.0368 ± 0.0005
Recall@20: 0.7361 ± 0.0099


In [ ]:


eval_rmse = rmse(test, all_predictions)
eval_mae = mae(test, all_predictions)
eval_rsquared = rsquared(test, all_predictions)
eval_exp_var = exp_var(test, all_predictions)

print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')

# factor 10
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

TOP_K=15
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

TOP_K =20
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')


# Manually calculate per-user ranking metrics to get std
per_user_map_list = []
per_user_ndcg_list = []
per_user_precision_list = []
per_user_recall_list = []
per_user_rmse_list = []
per_user_mae_list = []

unique_test_users = test['userID'].unique()

for k in [10,15,20]:
  for user_id in unique_test_users:
      test_user = test[test['userID'] == user_id]
      all_predictions_user = all_predictions[all_predictions['userID'] == user_id]

      # Filter predictions for the current user
      predictions_user = predictions[predictions['userID'] == user_id]

      # Ensure there are relevant entries for evaluation
      if not test_user.empty and not all_predictions_user.empty:
          # Compute ranking metrics for the user
          map_val = map_at_k(test_user, all_predictions_user, col_prediction='prediction', k=k)
          ndcg_val = ndcg_at_k(test_user, all_predictions_user, col_prediction='prediction', k=k)
          precision_val = precision_at_k(test_user, all_predictions_user, col_prediction='prediction', k=k)
          recall_val = recall_at_k(test_user, all_predictions_user, col_prediction='prediction', k=k)

          # Append only if the metric is not NaN (meaning computation was possible)
          if not pd.isna(map_val): per_user_map_list.append(map_val)
          if not pd.isna(ndcg_val): per_user_ndcg_list.append(ndcg_val)
          if not pd.isna(precision_val): per_user_precision_list.append(precision_val)
          if not pd.isna(recall_val): per_user_recall_list.append(recall_val)

      # Compute RMSE for the user if there are predictions
      if not test_user.empty and not predictions_user.empty:
          rmse_val = rmse(test_user, predictions_user)
          if not pd.isna(rmse_val): per_user_rmse_list.append(rmse_val)
          mae_val = mae(test_user, predictions_user)
          if not pd.isna(mae_val): per_user_mae_list.append(mae_val)


  # Convert lists to pandas Series to calculate mean and std
  per_user_map = pd.Series(per_user_map_list)
  per_user_ndcg = pd.Series(per_user_ndcg_list)
  per_user_precision = pd.Series(per_user_precision_list)
  per_user_recall = pd.Series(per_user_recall_list)
  per_user_rmse = pd.Series(per_user_rmse_list)
  per_user_mae = pd.Series(per_user_mae_list)

  # Calculate mean and std for per-user ranking metrics
  mean_map = per_user_map.mean() if not per_user_map.empty else 0.0
  std_map = per_user_map.std() if not per_user_map.empty else 0.0
  mean_ndcg = per_user_ndcg.mean() if not per_user_ndcg.empty else 0.0
  std_ndcg = per_user_ndcg.std() if not per_user_ndcg.empty else 0.0
  mean_precision = per_user_precision.mean() if not per_user_precision.empty else 0.0
  std_precision = per_user_precision.std() if not per_user_precision.empty else 0.0
  mean_recall = per_user_recall.mean() if not per_user_recall.empty else 0.0
  std_recall = per_user_recall.std() if not per_user_recall.empty else 0.0

  # Calculate mean and std for per-user RMSE
  mean_rmse_per_user = per_user_rmse.mean() if not per_user_rmse.empty else 0.0
  std_rmse_per_user = per_user_rmse.std() if not per_user_rmse.empty else 0.0
  mean_mea_per_user = per_user_mae.mean() if not per_user_mae.empty else 0.0
  std_mae_per_user = per_user_mae.std() if not per_user_mae.empty else 0.0

  # Calculate mean variance and standard deviation across users
  # First, merge test and predictions to calculate user-wise stats easily
  merged_test_predictions = pd.merge(test, predictions[['userID', 'itemID', 'prediction']], on=['userID', 'itemID'], how='left')

  user_wise_test_var = merged_test_predictions.groupby('userID')['rating'].var().mean()
  user_wise_test_std = merged_test_predictions.groupby('userID')['rating'].std().mean()
  user_wise_pred_var = merged_test_predictions.groupby('userID')['prediction'].var().mean()
  user_wise_pred_std = merged_test_predictions.groupby('userID')['prediction'].std().mean()

  print("Overall RMSE:\t\t%f" % eval_rmse,
        "Overall MAE:\t\t%f" % eval_mae,
        "Overall R-squared:\t%f" % eval_rsquared,
        "Overall Exp Var:\t%f" % eval_exp_var, sep='\n')

  print("Overall MAP:\t%f" % eval_map,
        "Overall NDCG:\t%f" % eval_ndcg,

        "Overall Precision@K:\t%f" % eval_precision,
        "Overall Recall@K:\t%f" % eval_recall, sep='\n')


  print('----')

  print("RMSE (Mean \u00B1 Std Per User):\t%f \u00B1 %f" % (mean_rmse_per_user, std_rmse_per_user), sep='\n')
  print("MEA (Mean \u00B1 Std Per User):\t%f \u00B1 %f" % (mean_mea_per_user, std_mae_per_user), sep='\n')
  print("MAP (Mean \u00B1 Std Per User):\t%f \u00B1 %f" % (mean_map, std_map),
        "NDCG (Mean \u00B1 Std Per User):\t%f \u00B1 %f" % (mean_ndcg, std_ndcg),
        "Precision@K (Mean \u00B1 Std Per User):\t%f \u00B1 %f" % (mean_precision, std_precision),
        "Recall@K (Mean \u00B1 Std Per User):\t%f \u00B1 %f" % (mean_recall, std_recall), sep='\n')

  print('---- Descriptive Statistics (Mean Across Users) ----')
  print("Mean User Test Ratings Variance:\t%f" % user_wise_test_var,
        "Mean User Test Ratings Std Dev:\t%f" % user_wise_test_std,
        "Mean User Prediction Variance:\t%f" % user_wise_pred_var,
        "Mean User Prediction Std Dev:\t%f" % user_wise_pred_std, sep='\n')

RMSE:		3.625281
MAE:		3.581325
rsquared:	-40.837241
exp var:	-0.008390
MAP:	0.182188
NDCG:	0.345088
Precision@K:	0.081549
Recall@K:	0.776868
MAP:	0.165868
NDCG:	0.298783
Precision@K:	0.087154
Recall@K:	0.614085
MAP:	0.182188
NDCG:	0.345088
Precision@K:	0.081549
Recall@K:	0.776868
Overall RMSE:		3.625281
Overall MAE:		3.581325
Overall R-squared:	-40.837241
Overall Exp Var:	-0.008390
Overall MAP:	0.182188
Overall NDCG:	0.345088
Overall Precision@K:	0.081549
Overall Recall@K:	0.776868
----
RMSE (Mean ± Std Per User):	3.590772 ± 0.648932
MEA (Mean ± Std Per User):	3.582836 ± 0.650291
MAP (Mean ± Std Per User):	0.140396 ± 0.186565
NDCG (Mean ± Std Per User):	0.234982 ± 0.238971
Precision@K (Mean ± Std Per User):	0.089295 ± 0.090668
Recall@K (Mean ± Std Per User):	0.415932 ± 0.393758
---- Descriptive Statistics (Mean Across Users) ----
Mean User Test Ratings Variance:	0.163173
Mean User Test Ratings Std Dev:	0.332110
Mean User Prediction Variance:	0.001829
Mean User Prediction Std Dev:	0.035